In [ ]:
!pip uninstall -y edge-rec
!pip install -e.

In [ ]:
from edge_rec.gaussian_diffusion import GaussianDiffusion, Trainer
from edge_rec.model import GraphReconstructionModel

model = GraphReconstructionModel.default()

diffusion_model = GaussianDiffusion(model, image_size=10)
trainer = Trainer(diffusion_model, folder="./movie_lens", train_num_steps=int(1e4))
print("Using device:", trainer.device)

In [ ]:
trainer.train()